In [8]:
from river_torch.anomaly import Autoencoder
from river import metrics
from river.datasets import CreditCard
from torch import nn
import math
from river.compose import Pipeline
from river.preprocessing import MinMaxScaler

In [9]:
dataset = CreditCard().take(5000)
metric = metrics.ROCAUC(n_thresholds=50)

def get_fc_ae(n_features):
    latent_dim = math.ceil(n_features / 2)
    return nn.Sequential(
        nn.Linear(n_features, latent_dim),
        nn.SELU(),
        nn.Linear(latent_dim, n_features),
        nn.Sigmoid(),
    )
ae = Autoencoder(build_fn=get_fc_ae, lr=0.005)
scaler = MinMaxScaler()
model = Pipeline(scaler, ae)

for x, y in dataset:
    score = model.score_one(x)
    model.learn_one(x=x)
    metric.update(y, score)
print(f"ROCAUC: {metric.get()}")

ROCAUC: 0.9064438663197919
